In [10]:
import torch, struct, os, psutil, subprocess, time, threading
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf

### Load and Evaluate

In [11]:
# TODO: use json file for model (results, config etc) management
arch_folder = "input-dense-dense/"
model_name = "784_56_10"
model_path = "../../models/"

state_dict = torch.load(model_path + arch_folder+ model_name + ".pth")
list_vars = state_dict

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to PyTorch tensors
test_images_pt = torch.tensor(test_images).float()
# Flatten and normalize the images
test_images_pt = test_images_pt.view(-1, 28*28) / 255.0  # Flatten and normalize


FileNotFoundError: [Errno 2] No such file or directory: '../../models/input-dense-dense/784_56_10.pth'

In [7]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 56)  # Flatten 28*28 and feed into 56 neurons
        self.fc2 = nn.Linear(56, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

### Debug

In [8]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Convert to PyTorch format [batch_size, total pixels]
# Since images are already normalized and flattened for TensorFlow, we can use the same arrays
train_images_pt = torch.tensor(train_images_tf).float()
test_images_pt = torch.tensor(test_images_tf).float()
train_labels_pt = torch.tensor(train_labels)
test_labels_pt = torch.tensor(test_labels)

In [9]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset for test data
test_dataset = TensorDataset(test_images_pt, test_labels_pt)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_pytorch_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_loader)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')


Accuracy of the PyTorch model on the test images: 63.90000000%


### Setup

In [143]:
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
os.makedirs(folder, exist_ok=True)

In [144]:
fname_out = "./bin/" + arch_folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"

os.makedirs("./bin/" + arch_folder, exist_ok=True)

fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (56, 784)
Processing variable: fc1.bias with shape:  (56,)
Processing variable: fc2.weight with shape:  (10, 56)
Processing variable: fc2.bias with shape:  (10,)


In [145]:
def load_img_from_file(data_file="input", show=False):
    try:
        with open(data_file, 'rb') as file:
            buf = file.read()
    except Exception as e:
        print(e)
        return None, e

    digits = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    
    if show:
        c = ""
        for row in range(28):
            for col in range(28):
                if buf[row * 28 + col] > 230:
                    c += "&"
                else:
                    c += "-"
            c += "\n"
        print(c)

    return digits, None

def save_img_to_file(image, data_file = "input"):
    try:
        # Convert to bytes
        image_bytes = image.astype('uint8').tobytes()
        
        # Write to file
        with open(data_file, 'wb') as file:
            file.write(image_bytes)
        
        #print(f"Image saved to {data_file}")
    except Exception as e:
        print(f"Error saving image: {e}")


In [146]:
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

### Test

In [179]:
ind = 56
img_out_path = folder + str(ind)
save_img_to_file(test_images[ind], img_out_path)

In [180]:
vm_file = "./bin/vm"
program = "./bin/gglm.bin"
model_in_path = fname_out

command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

In [181]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")

# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

stderr[-2]

Process ID: 767774


'4'

In [182]:
predicted_labels[ind]

4

In [183]:
# Example usage
digits, error = load_img_from_file(img_out_path,show=True)

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
---------&------------------
---------&&-----------------
--------&&&---------&&------
--------&&----------&&------
--------&&----------&&------
--------&&---------&&-------
--------&&---------&&-------
--------&&---------&&-------
-------&&&--------&&--------
-------&&---------&&--------
-------&&&&&&&&&&&&&--------
-------&&&&&&&--&&&&--------
------------------&---------
-----------------&&---------
-----------------&&---------
-----------------&&---------
----------------&&&---------
----------------&&&---------
----------------&&----------
-----------------&----------
----------------------------
----------------------------
----------------------------



### Benchmark

In [113]:
import concurrent.futures

def benchmark(test_images, tmp_folder, model_in_path, vm_file = "./bin/vm", program = "./bin/gglm.bin", threaded = True):
    benchmark_start_time = time.time()
    veri_infer = []
    mem_usage = []
    time_cost = []
    for ind, img in enumerate(test_images):
        img_out_path = tmp_folder + str(ind)
        save_img_to_file(img, img_out_path)

        # Exclusion of Pre-processing
        start_time = time.time()
        command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]
        
        print ("Process for image", ind)
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Get the process ID
        pid = process.pid

        if threaded:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                future = executor.submit(monitor_memory, pid)
                _, stderr = process.communicate()
                max_memory = future.result()
                #print(f"Maximum memory used in multi-threaded mode: {max_memory} bytes")
        else:
            max_memory = monitor_memory(pid)  # Run in the same thread
            _, stderr = process.communicate()
            #print(f"Maximum memory used in single-threaded mode: {max_memory} bytes")
        veri_infer.append(int(stderr[-2]))
        mem_usage.append(max_memory)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    #print ("total mem:", sum(mem))
    return veri_infer, mem_usage, time_cost

In [156]:
veri_infer, mem_usage, time_cost = benchmark(test_images[:20], './tmp/', model_in_path)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Process for image 19
Total time: 77.32435870170593


### Save Results

In [157]:
def calculate_loss(veri_infer, predicted_labels):
    count = 0
    for i in range(len(veri_infer)):
        if veri_infer[i] != predicted_labels[i]:
            count +=1
            print (f"Index {i} Not match!")

    return count/len(veri_infer)*100

In [158]:
calculate_loss(veri_infer, predicted_labels)

Index 9 Not match!
Index 16 Not match!


10.0

In [152]:
import pandas as pd

csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [153]:
new_row = {
    'Framework': ['opml (pytorch)'],
    'Architecture': ['Input-Dense-Dense (784 * 56 * 10, w/ relu)'],
    '# Layers': [3],
    '# Parameters': [44543],
    'Testing Size': [len(veri_infer)],
    'Accuracy Loss (%)': [calculate_loss(veri_infer, predicted_labels)],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


Index 9 Not match!
Index 16 Not match!
Index 20 Not match!
Index 23 Not match!
Index 25 Not match!
Index 33 Not match!
Index 34 Not match!
Index 45 Not match!
Index 52 Not match!
Index 53 Not match!
Index 55 Not match!
Index 58 Not match!
Index 62 Not match!
Index 63 Not match!
Index 66 Not match!
Index 73 Not match!
Index 78 Not match!
Index 92 Not match!
Index 105 Not match!
Index 108 Not match!
Index 118 Not match!
Index 119 Not match!
Index 120 Not match!
Index 126 Not match!
Index 129 Not match!
Index 136 Not match!
Index 149 Not match!
Index 150 Not match!
Index 151 Not match!
Index 152 Not match!
Index 153 Not match!
Index 155 Not match!
Index 158 Not match!
Index 165 Not match!
Index 175 Not match!
Index 182 Not match!
Index 185 Not match!
Index 187 Not match!
Index 194 Not match!
Index 206 Not match!
Index 209 Not match!
Index 211 Not match!
Index 214 Not match!
Index 215 Not match!
Index 217 Not match!
Index 218 Not match!
Index 219 Not match!
Index 233 Not match!
Index 234 N

In [154]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)


In [155]:
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,250,0.4,88.998094,2.285579,3.655122,0.440126
1,opml (pytorch),"Input-Dense-Dense (784 * 56 * 10, w/ relu)",3,44543,1000,20.9,89.122078,2.247846,3.664727,0.433921
